## Objective
Create the Key statistics dashboard and graphs for chapter "The popularity of SBTi"


# Importing, loading data, visuals and error corrections

In [2]:
#Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import seaborn as sns
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, HoverTool, Slider, CustomJS, Legend, LegendItem
from bokeh.layouts import column
from bokeh.palettes import Category20
from bokeh.palettes import TolRainbow
from bokeh.transform import factor_cmap
from bokeh.plotting import output_notebook

from itertools import cycle
# Ensure Bokeh plots are displayed in the notebook
output_notebook()

Loading BokehJS ...

In [3]:
#Font properties for the plots
#font_prop = font_manager.FontProperties(fname='C:\\Windows\\Fonts\\source-sans-pro.ttf')
font_reg = font_manager.FontProperties(fname="C:/Fonts/SourceSansPro-Regular.otf")
font_bold = font_manager.FontProperties(fname="C:/Fonts/SourceSansPro-Bold.otf")
font_italic = font_manager.FontProperties(fname="C:/Fonts/SourceSansPro-It.ttf")
font_light = font_manager.FontProperties(fname="C:/Fonts/SourceSansPro-Light.otf")

In [4]:
#colors for the plots
sbt_palette4 = [
    # Pastel main colors
    "#B3D4E0",  # Pastel Light Blue
    "#146A4A",  # Deep Green
    "#E0C15A",  # Warm Yellow
    "#E65A70",  # Coral Pink
    "#4B2C69",  # Deep Purple

    # Pastel extended palette
    "#A8E1E3",  # Light Aqua
    "#2A7764",  # Medium Teal
    "#F0DAA5",  # Pale Yellow
    "#CC9E2F",  # Rich Mustard
    "#7EB6CE",  # Sky Blue
    "#104232",  # Forest Green
    "#F4989C",  # Soft Coral
    "#6B3F7D",  # Indigo Purple
    "#D7E4CC",  # Pale Sage
    "#D97158",  # Terracotta
    "#D2ECF2",  # Powder Blue
    "#2B4C6F",  # Navy Blue
    "#9BD3AE",  # Mint Green
    "#7A8B8D",  # Slate Gray
    "#E8C68B",   # Warm Sand
    
       # New additional colors
    "#FFB6C1",  # Light Pink
    "#FFD700",  # Gold
    "#8A2BE2",  # Blue Violet
    "#98FB98"   # Pale Green
]


In [5]:
# Load the data
filepath1 = r'ByCompany.csv'
Comp = pd.read_csv(filepath1, sep=';')
filepath2 = r'ByTargets.csv'
Targ = pd.read_csv(filepath2, sep=';')

In [6]:
# Mapping of sectors to categories
sector_category_map = {
    'Ground Transportation - Railroads Transportation': 'Transportation',
    'Semiconductors and Semiconductors Equipment': 'Technology',
    'Air Freight Transportation and Logistics': 'Transportation',
    'Trading Companies and Distributors, and Commercial Services and Supplies': 'Commercial Services',
    'Water Transportation - Water Transportation': 'Transportation',
    'Food and Beverage Processing': 'Food and Beverage',
    'Telecommunication Services': 'Telecommunication',
    'Construction and Engineering': 'Construction',
    'Pharmaceuticals, Biotechnology and Life Sciences': 'Healthcare',
    'Professional Services': 'Professional Services',
    'Containers and Packaging': 'Packaging',
    'Real Estate': 'Real Estate',
    'Consumer Durables, Household and Personal Products': 'Consumer Products',
    'Electric Utilities and Independent Power Producers and Energy Traders (including Fossil, Alternative and Nuclear Energy)': 'Energy',
    'Retailing': 'Retail',
    'Mining - Iron, Aluminum, Other Metals': 'Mining',
    'Textiles, Apparel, Footwear and Luxury Goods': 'Consumer Products',
    'Software and Services': 'Technology',
    'Hotels, Restaurants and Leisure, and Tourism Services': 'Hospitality',
    'Electrical Equipment and Machinery': 'Manufacturing',
    'Food Production - Agricultural Production': 'Food Production',
    'Building Products': 'Construction',
    'Food and Staples Retailing': 'Retail',
    'Automobiles and Components': 'Automotive',
    'Forest and Paper Products - Forestry, Timber, Pulp and Paper, Rubber': 'Forestry and Paper Products',
    'Tobacco': 'Tobacco',
    'Solid Waste Management Utilities': 'Utilities',
    'Chemicals': 'Chemicals',
    'Construction Materials': 'Construction',
    'Technology Hardware and Equipment': 'Technology',
    'Mining - Other (Rare Minerals, Precious Metals and Gems)': 'Mining',
    'Water Utilities': 'Utilities',
    'Homebuilding': 'Construction',
    'Ground Transportation - Highways and Railtracks': 'Transportation',
    'Food Production - Animal Source Food Production': 'Food Production',
    'Healthcare Equipment and Supplies': 'Healthcare',
    'Air Transportation - Airport Services': 'Transportation',
    'Media': 'Media',
    'Banks, Diverse Financials, Insurance': 'Financial Services',
    'Ground Transportation - Trucking Transportation': 'Transportation',
    'Specialized Consumer Services': 'Consumer Services',
    'Specialized Financial Services, Consumer Finance, Insurance Brokerage Firms': 'Financial Services',
    'Healthcare Providers and Services, and Healthcare Technology': 'Healthcare',
    'Water Transportation - Ports and Services': 'Transportation',
    'Air Transportation - Airlines': 'Transportation',
    'Aerospace and Defense': 'Defense',
    'Tires': 'Automotive',
    'Education Services': 'Education',
    'Gas Utilities': 'Utilities'
}

# Function to map sectors to categories
def map_category(sector):
    return sector_category_map.get(sector, 'Unknown')
# Apply the function to the 'sector' column
Targ['sectorCat'] = Targ['sector'].apply(map_category)
Comp['sectorCat'] = Comp['sector'].apply(map_category)
# Display the DataFrame
Targ['sectorCat'].unique()
# Place sectorCat next to sector in the DataFrame
Targ = Targ[['sector', 'sectorCat'] + [col for col in Targ.columns if col not in ['sector', 'sectorCat']]]
Comp = Comp[['sector', 'sectorCat'] + [col for col in Comp.columns if col not in ['sector', 'sectorCat']]]

In [7]:
#make Targ['date_published'] a datetime object
Targ['date_published'] = pd.to_datetime(Targ['date_published'], errors='coerce')
#seperate year from date_published
Targ['Comitted year'] = Targ['date_published'].dt.year
#sort targ by year
Targ = Targ.sort_values(by='Comitted year', ascending=True)
# limit to 2015-2024
Targ = Targ[Targ['Comitted year'].between(2015, 2024)]
#check if year is sorted correctly
Targ['Comitted year'].unique()

array([2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024])

In [8]:
# Remove duplicates of stbi_id in Targ to make counting easier
#Targ = Targ.drop_duplicates(subset=['sbti_id'])

In [9]:
#Adjust the date in the Company data
Comp['date_updated'] = pd.to_datetime(Comp['date_updated'], errors='coerce')
#isolating year in Comp
Comp['year_updated'] = Comp['date_updated'].dt.year
Comp=Comp.sort_values(by='year_updated', ascending=True)

C:\Users\gfjgu\AppData\Local\Temp\ipykernel_16652\2031487950.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  Comp['date_updated'] = pd.to_datetime(Comp['date_updated'], errors='coerce')


In [10]:
Comp.head()

,sector,sectorCat,sbti_id,company_name,organization_type,location,region,near_term_status,near_term_target_classification,near_term_target_year,...,long_term_target_year,net_zero_status,net_zero_year,reason_for_extension_or_removal,date_updated,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,year_updated
4243,"Banks, Diverse Financials, Insurance",Financial Services,40008661,Teachers Mutual Bank,Financial Institution,Australia,Oceania,Commitment removed,NaN,NaN,...,NaN,NaN,NaN,Expired commitment,2015-11-01,NaN,NaN,NaN,NaN,2015
4300,Ground Transportation - Railroads Transportation,Transportation,40005752,Thalys,Corporate,Belgium,Europe,Targets set,Well-below 2°C,2020,...,NaN,NaN,NaN,NaN,2015-10-30,NaN,NaN,NaN,NaN,2015
527,"Banks, Diverse Financials, Insurance",Financial Services,40007348,AXA Group,Financial Institution,France,Europe,Committed,NaN,NaN,...,NaN,NaN,NaN,Commitment Compliance Policy transitory grace ...,2015-06-01,NaN,NaN,NaN,NaN,2015
573,"Banks, Diverse Financials, Insurance",Financial Services,40013764,Bank J. Safra Sarasin AG,Financial Institution,Switzerland,Europe,Committed,NaN,NaN,...,NaN,NaN,NaN,FINZ extension policy,2015-06-01,NaN,NaN,NaN,NaN,2015
2206,"Banks, Diverse Financials, Insurance",Financial Services,40005141,ING,Financial Institution,Netherlands,Europe,Committed,NaN,NaN,...,NaN,NaN,NaN,FINZ extension policy,2015-06-01,NaN,NaN,NaN,NaN,2015


In [11]:
Targ.head()

,sector,sectorCat,sbti_id,company_name,location,region,organization_type,action,company_temperature_alignment,target,scope,target_value,type,sub_type,target_classification_short,base_year,target_year,date_published,Comitted year
9915,Ground Transportation - Railroads Transportation,Transportation,40005752,Thalys,Belgium,Europe,Corporate,Target,Well-below 2°C,Near-term,1+2+3,"41,40%",Intensity,Reduction,Well-below 2°C,2008,2020,2015-10-30,2015
9916,Ground Transportation - Railroads Transportation,Transportation,40005752,Thalys,Belgium,Europe,Corporate,Target,Well-below 2°C,Near-term,3,"50,00%",Engagement,Other,NaN,2017,2020,2015-10-30,2015
506,Semiconductors and Semiconductors Equipment,Technology,40015387,"Advanced Micro Devices, Inc",United States of America,Northern America,Corporate,Target,Well-below 2°C,Near-term,3,"0,4",Engagement,Other,NaN,2014,2020,2016-06-27,2016
505,Semiconductors and Semiconductors Equipment,Technology,40015387,"Advanced Micro Devices, Inc",United States of America,Northern America,Corporate,Target,Well-below 2°C,Near-term,3,30%,Engagement,Other,NaN,2014,2020,2016-06-27,2016
5137,Air Freight Transportation and Logistics,Transportation,40010493,International Post Corporation (IPC),Belgium,Europe,Corporate,Target,2°C,Near-term,1+2+3,20%,Intensity,Reduction,2°C,2013,2025,2016-02-05,2016


# Key statistical dashboard
We want to make the following key numbers and present them with a nice finish
1. Total companies with SBTi targets validated by the SBTi by the end of 2024
2. Increase in the number of companies with SBTi targets by the end of 2024 compared to 2023
3. Nr of companies set SBTi targets for the first time in 2024
4. Increase in companies with net-zero targets in 2024
5. Total accumulative expired targets
6. Total companies that have retracted their targets

In [17]:
# Count number of commitments in company data --------- 1
Comp['commitments'] = Comp.groupby('company_name')['company_name'].transform('count')

# Count the increase in commitments between 2023 and 2024 in company data
Comp['commitments_2023'] = Comp[Comp['year_updated'].isin([2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023])]['commitments']
#Comp['commitments_2024'] = Comp[Comp['year_updated'] == 2024]['commitments'] # ------------ 3

# Calculate the % increase in commitments between 2023 and 2024 # ------------ 2
Pincrease24 = (Comp['commitments'] / Comp['commitments_2023']) * 100

# Count net zero targets in target data
Targ['net_zero'] = Targ[Targ['target'] == 'Net-zero'].groupby('sbti_id')['sbti_id'].transform('count')

# Count net zero targets in 2024 in target data
Targ['net_zero_24'] = Targ[(Targ['target'] == 'Net-zero') & (Targ['Comitted year'] == 2024)].groupby('sbti_id')['sbti_id'].transform('count')

#Count companies with expired commitments ------------- 5
Comp['Expired'] = Comp[Comp['reason_for_extension_or_removal'] == 'Expired commitment'].groupby('sbti_id')['sbti_id'].transform('count')

# Count companies wih retracted commitments ------------- 6
Comp['Withdrawn'] = Comp[Comp['reason_for_extension_or_removal'] == 'Withdrawn commitment'].groupby('sbti_id')['sbti_id'].transform('count')

# Highlighting 6 key values
# Storing the values in a dictionary for easy access
Comp_values = {
    'commitments': Comp['commitments'].sum(),
    'Pincrease24': Pincrease24.sum(),
    'commitments_2024': Comp['commitments_2024'].sum(),
    'net_zero': Targ['net_zero'].sum(),
    'net_zero_2024': Targ['net_zero_24'].sum(),
    'expired': Comp['Expired'].sum(),
    'retracted': Comp['Withdrawn'].sum()
}
# Display the values
for key, value in Comp_values.items():
    print(f"{key}: {value}")


commitments: 10889
Pincrease24: 555200.0
commitments_2024: 4506.0
net_zero: 1430.0
net_zero_2024: 867.0
expired: 943.0
retracted: 33.0


# The graphs for chapter "The popularity of SBTi"
Graph answering the questions:
* When did it start?
* How has it grown?
* How has popularity grown based on organisation type (SME, corporation, financial etc)?

Graphs answering the questions:
* What countries and industry sectors were the first to register?
* With which countries and industries is SBTi most popular?
* What industries are most popular in each european country?

In [ ]:
# Plotting the accumulated number of commitments by year
# Count the number of commitments per year
commitments_per_year = Targ.groupby('Comitted year').size().reset_index(name='commitments')
# Create a cumulative sum of commitments
commitments_per_year['cumulative_commitments'] = commitments_per_year['commitments'].cumsum()
# Create a Bokeh plot
output_file("commitments_per_year.html")
p = figure(title="Cumulative Commitments by Year", x_axis_label='Year', y_axis_label='Cumulative Commitments', width=800, height=400)
p.line(commitments_per_year['Comitted year'], commitments_per_year['cumulative_commitments'], line_width=2, color='blue', legend_label='Cumulative Commitments')
p.circle(commitments_per_year['Comitted year'], commitments_per_year['cumulative_commitments'], size=8, color='red', legend_label='Commitments')
p.legend.location = "top_left"
# Plot the figure
show(p)